# Processing I: from XDF to raw files

As described in the preregistration of methods (@REF), the data has been collected using LabStreamLayer, allowing for recording of precisely synchronized streams. These streams include:
- MicStream (16kHz)
- WebcamFrameStream (ca. 60 Hz)
- MarkerStream (sent from custom buttonbox)
- BalanceBoardStream (500 Hz)

LSL outputs single .xdf file that contains all the data streams. This script serves for extracting the streams and creating raw files for further processing.


In [ ]:
# import packages

import os
import pyxdf
import glob
import pandas as pd
import numpy as np
import wave, struct, math, random
from scipy.io import wavfile
import noisereduce as nr
import json
import pandas as pd
import cv2
import random
import json

#from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip #this is the video clipping function
#from moviepy.video.io.VideoFileClip import VideoFileClip #alternative for snipping video
#from moviepy.editor import VideoFileClip, AudioFileClip, CompositeAudioClip

# set folders
curfolder = os.getcwd()
datafolder = curfolder+'\\data\\'
errorlogs = datafolder+'\\error_logs\\'
experiment_to_process = '\\raw_data\\'
outputfolder = datafolder+'\\Data_processed\\CsvDataTS_raw\\' # outputfolder raw
trialfolder = datafolder+'\\Data_processed\\Data_trials\\' # outputfolder trialbased
targetfolder = datafolder+experiment_to_process # this is where raw data is stored

# map all the folders in the target folder
datafolders = glob.glob(targetfolder+'*\\')

# extract the folder IDs
datafolders_id = [x.split('\\')[-2] for x in datafolders]

# print(curfolder)
# print(targetfolder)
print(datafolders_id)

# identify all xdf files and all the associated triallist info
xdffiles = []
trialdatas = []

for i in datafolders_id:
    file = glob.glob(targetfolder+i+'\\*.xdf')
    trialfile = targetfolder+i+'\\'+i+'_results'+'.csv'
    trialdatas.append(trialfile)
    xdffiles.extend(file)

# these are the xdf files we need to process
print(xdffiles)
print(trialdatas)

c:\Users\kadava\AppData\Local\anaconda3\envs\TSPROCESS\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['0_1', '0_2']
['e:\\FLESH_ContinuousBodilyEffort\\01_XDF_processing\\data\\\\raw_data\\0_1\\session_S001.xdf', 'e:\\FLESH_ContinuousBodilyEffort\\01_XDF_processing\\data\\\\raw_data\\0_2\\session_S002.xdf']
['e:\\FLESH_ContinuousBodilyEffort\\01_XDF_processing\\data\\\\raw_data\\0_1\\0_1_results.csv', 'e:\\FLESH_ContinuousBodilyEffort\\01_XDF_processing\\data\\\\raw_data\\0_2\\0_2_results.csv']


# Extracting streams from XDF file

Using `pyxdf` package, we can extract the streams from the XDF file.

First, we extract streams for the whole session. Then, we cut these streams into trial-sized chunks, based on the marker stream that indicates the start of each trial. Moreover, we connect the trial with information from the PsychoPy log file, which includes metadata about the trial (i.e., concept, correction number, etc.)

(Note that this code takes some time to execute.)


In [7]:
# audio write function
def to_audio(fileloc, timeseries, samplerate = 16000, channels = 1):
    if timeseriestype == 'Mic': 
        obj = wave.open(fileloc,'w')
        obj.setnchannels(channels) # mono
        obj.setsampwidth(2)
        obj.setframerate(float(samplerate))
        for i in timeseries:
            data = struct.pack('<h', int(i[0]))
            obj.writeframesraw( data )
        obj.close()

# function to retrieve closest value
def find_closest_value_and_retrieve(df, target_number, target_column, retrieve_column):
    # get the absolute differences between a value in column and the target number
    differences = abs(df[target_column] - target_number)
    
    # find the index of the minimum difference
    min_difference_index = differences.idxmin()
    
    # retrieve the closest value in the target column
    closest_value = df.loc[min_difference_index, target_column]
    #print(closest_value)
    # retrieve the corresponding value from the column
    result_value = df.loc[min_difference_index, retrieve_column]

    return result_value

In [ ]:
errorlist = []

for dat in datafolders_id:
    print('Loading in data from participant: ' + dat)
    trialdata = pd.read_csv(targetfolder+dat+'\\'+dat+'_results.csv', sep=",")
    #print(trialdata)
    
    # get the xdf file
    files = glob.glob(targetfolder+dat+'\\*.xdf')
    streams, header = pyxdf.load_xdf(files[0])   
    # we go through each stream and save it as a csv, 
    # if it's Mic stream, also as a .wav file
    for stream in streams:
        timeseriestype = stream['info']['name'][0]
        print(timeseriestype)
        samplerate = round(float(stream['info']['nominal_srate'][0]))
        # in the xdf loop over the streams and save it as csv if not yet exists
        channelcount = stream['info']['channel_count'][0]
        print('working on stream: ' + timeseriestype + '  with a channel count of ' + str(channelcount) +'\n and a sampling rate of ' + str(samplerate))
        timevec = stream['time_stamps']
        timeseries = stream['time_series']
        matrix_aux = np.vstack([np.transpose(timevec),np.transpose(timeseries)])
        matrix = np.transpose(matrix_aux)
        df_lab = pd.DataFrame(matrix)
        df_lab.to_csv(outputfolder+dat+'_'+timeseriestype+'_nominal_srate'+str(samplerate)+'.csv',index=False)
        #for audio data also create a wav file
        if timeseriestype == 'Mic':
            wavloc = outputfolder+'Audio/'+dat+'_'+timeseriestype+'_nominal_srate'+str(samplerate)+'.wav'
            to_audio(wavloc, timeseries)
            # load data
            rate, data = wavfile.read(wavloc)
            # perform noise reduction
            reduced_noise = nr.reduce_noise(y=data, sr=rate, n_std_thresh_stationary=1.5,stationary=True)
            wavloc2 = outputfolder+'Audio/'+dat+'_'+timeseriestype+'_nominal_srate'+str(samplerate)+'_denoised.wav'
            wavfile.write(wavloc2, rate, reduced_noise)

        print('done with processing a complete time series and audio data')
        print('we will now start making trial snipped data') 

        # load the MyMarkerStream
        markers = pd.read_csv(outputfolder+dat+'_MyMarkerStream_nominal_srate0.csv')
        
        # cut all timeseries to trial level based on the markers
        if timeseriestype != 'MyMarkerStream':
            
            beginlist = []
            endlist = []
            timestamps = []
            timestamps_2 = []
            tpose_starts = []
            tpose_ends = []

            # iterate over markers and save times of trial starts and ends
            for row in markers.iterrows():
                # Accessing positional elements explicitly with .iloc
                if 'Trial_start' in row[1].iloc[1] or 'Practice trial starts' in row[1].iloc[1]:
                    beginlist.append(row[1].iloc[0])
                if 'Trial_end' in row[1].iloc[1] or 'Practice trial ends' in row[1].iloc[1]:
                    endlist.append(row[1].iloc[0])
                if 'Experiment_start' in row[1].iloc[1]:
                    timestamps.append(row[1].iloc[0])
                if 'New block starts' in row[1].iloc[1]:
                    timestamps_2.append(row[1].iloc[0])
                if 'Tpose starts' in row[1].iloc[1]:
                    tpose_starts.append(row[1].iloc[0])
                if 'Tpose ends' in row[1].iloc[1]:
                    tpose_ends.append(row[1].iloc[0])
                    
            # converting coefficient for lsl to psychopy time
            exp_start_pp = float(trialdata['exp_start'][0])
            block_start_pp = float(trialdata['block_start'][0])
            # get to lsl_to_pp coefficient
            if timestamps != []:
                lsl_to_pp = timestamps[0] - exp_start_pp
            else:
                lsl_to_pp = timestamps_2[0] - block_start_pp
        
            # now we can proceed to cutting   
            for i in range(len(beginlist)):
                # prepare the range of the trial
                begin = beginlist[i]
                end = endlist[i]
                indices = (df_lab.loc[:,0] > begin) & (df_lab.loc[:,0] < end)
                beginst = min(df_lab.loc[:,0]) #start time of the timeseries
                endst = max(df_lab.loc[:,0])  #end time of the timeseries
                subset = df_lab.loc[indices, :]
                # convert the beginst to psychopy time
                beginst_pp = begin - lsl_to_pp
                # now find in trialdata the closest time to the beginst_pp to gather info
                # whether it is practice or trial
                practice = find_closest_value_and_retrieve(trialdata, beginst_pp, 'trial_start', 'practice')
                if practice == 'practice':
                    trialtype = 'pr'
                else:
                    trialtype = 'trial'
                
                # which participant it is
                cycle = find_closest_value_and_retrieve(trialdata, beginst_pp, 'trial_start', 'cycle')
                if cycle == 0:
                    participant = 'p0'
                else:
                    participant = 'p1'

                # what concept it is
                word = find_closest_value_and_retrieve(trialdata, beginst_pp, 'trial_start', 'word')
                # modality
                modality = find_closest_value_and_retrieve(trialdata, beginst_pp, 'trial_start', 'modality')
                # correction, if applicable
                correction_info = find_closest_value_and_retrieve(trialdata, beginst_pp, 'trial_start', 'correction')
                if correction_info == 0:
                    correction = '_c0'
                elif correction_info == 1:
                    correction = '_c1'
                elif correction_info == 2:
                    correction = '_c2'
                else:
                    correction = ''
                
                # continue saving
                if(len(subset.axes[0])<2):
                    errorlist.append(dat + " for "+ timeseriestype + " for trial " + str(i) + 'NO DATA WITHIN RANGE...')
                if(len(subset.axes[0])>2):
                     # save subset to csv
                      subset.to_csv(trialfolder+dat+'_'+trialtype+'_'+ str(i) +'_'+timeseriestype+'_nominal_srate'+str(samplerate)+'_'+participant+'_'+word+'_'+modality+correction+'.csv', index=False)
                      if timeseriestype == 'Mic':
                            wavloc = trialfolder+'Audio/'+dat+'_'+trialtype+'_'+ str(i) +'_'+timeseriestype+'_nominal_srate'+str(samplerate)+'_'+participant+'_'+word+'_'+modality+correction+'.wav'
                            to_audio(wavloc, timeseries[indices])
                          # also apply denoising
                            reduced_noiseclip = reduced_noise[indices]
                            wavloc2 = trialfolder+'Audio/'+dat+'_'+trialtype+'_'+ str(i) +'_'+timeseriestype+'_nominal_srate'+str(samplerate)+'_'+participant+'_'+word+'_'+modality+correction+'_denoised.wav'
                            wavfile.write(wavloc2, rate, reduced_noiseclip)
            
            # get information about the tpose for camera
            if timeseriestype == 'MyWebcamFrameStream':
                for i in range(len(tpose_starts)):
                    begin = tpose_starts[i]
                    end = tpose_ends[i]
                    indices = (df_lab.loc[:,0] > begin) & (df_lab.loc[:,0] < end)
                    beginst = min(df_lab.loc[:,0])
                    endst = max(df_lab.loc[:,0])
                    subset = df_lab.loc[indices, :]
                    # save subset to csv
                    subset.to_csv(trialfolder+dat+'_'+'tpose_'+ str(i) +'_'+timeseriestype+'_nominal_srate'+str(samplerate)+'.csv', index=False)
        
        # after every participants well save the error log
        errors = pd.DataFrame(errorlist, columns=['file_error'])
        file_path = errorlogs+'error_log_cuttingtrails.csv'
        errors.to_csv(file_path, index=False) 
        
print('Were done: proceed to snipping videos to triallevel')

Loading in data from participant: 0_1
MyMarkerStream
working on stream: MyMarkerStream  with a channel count of 1
 and a sampling rate of 0
done with processing a complete time series and audio data
we will now start making trial snipped data
MyWebcamFrameStream
working on stream: MyWebcamFrameStream  with a channel count of 1
 and a sampling rate of 500
done with processing a complete time series and audio data
we will now start making trial snipped data
BalanceBoard_stream
working on stream: BalanceBoard_stream  with a channel count of 4
 and a sampling rate of 500
done with processing a complete time series and audio data
we will now start making trial snipped data
Mic
working on stream: Mic  with a channel count of 1
 and a sampling rate of 16000
done with processing a complete time series and audio data
we will now start making trial snipped data


Now we have raw data for each trial in csv and wav, but we still need to write videos for each trial. We will use the trial-cut files to access the range of frames in the original raw video file and cut it out from it


In [ ]:
errorlist = []

# this is a folder with timeseries
tsfolder = datafolder+'\\Data_processed\\Data_trials\\'
# this is where the raw long video is
videofolder = datafolder+'raw_data\\'
#print(tsfolder)
#print(videofolder)

# loop through the csv's in tsfolder that has string 'MyWebcamStream' in name
for file in os.listdir(tsfolder):
    print(file)
    if 'MyWebcamFrameStream' in file:
        print('Now processing file '+ file)
        # if it is a tpose file, we skip it for now
        if 'tpose' in file:
            continue

        # the name looks like this 0_1_trial_0_MyWebcamFrameStream_nominal_srate500_p0_bitter_geluiden.csv

        dyadIndex = file.split('_')[0]   # this is dyad number
        partIndex = file.split('_')[1]   # this is part of the session
        sessionIndex = dyadIndex + '_' + partIndex # this is the session index
        trialIndex = file.split('_')[3] # this is trial number
        participant = file.split('_')[7] # this is participant 0/1
        word = file.split('_')[8] # this is the concept
        modality = file.split('_')[9].split('.')[0] # this is the modality

        # assess the correction
        if 'c0' in file:
            correction = '_c0'
        elif 'c1' in file:
            correction = '_c1'
        elif 'c2' in file:
            correction = '_c2'
        else:
            correction = ''


        # assess the trial type
        if 'pr' in file:
            trialtype = 'pr'
        else:
            trialtype = 'trial'

        trialdata = pd.read_csv(tsfolder+file)
        #print(trialdata)
        videolong = videofolder+sessionIndex+'\\'+sessionIndex+'-video.avi' # this is the long video 
        begin_time = trialdata['0'].min() # begin time of the trial
        #print(begin_time)
        end_time = trialdata['0'].max() # end time of the trial
        #print(end_time)
        # get the begin and end frame
        begin_frame = trialdata['1'].min().astype(int)
        #print(begin_frame)
        end_frame = trialdata['1'].max().astype(int)
        #print(end_frame)
        totframes = end_frame-begin_frame # total number of frames in the trial
        #print(totframes)
        frames = range(begin_frame, end_frame) # get all the frames in trial
        #print(frames)
        # load in the long video
        print('Loading the original video')
        capture = cv2.VideoCapture(videolong) 
        
        # some meta-info about the video
        #frameWidth = capture.get(ffmpegcv.CAP_PROP_FRAME_WIDTH)
        #frameHeight = capture.get(ffmpegcv.CAP_PROP_FRAME_HEIGHT)
        #frate =  capture.get(ffmpegcv.CAP_PROP_FPS)
        #print('frame rate: '+str(frate))
        #fourcc = ffmpegcv.VideoWriter_fourcc(*'M','J','P','G') #for different video formats you could use e.g., *'XVID'

        # what is the original fps of the video
        originalfps = round((totframes/(end_time-begin_time)),3)
        print('original fps: '+str(originalfps))
        
        # this is the location where the video will be saved
        vidloc = trialfolder+sessionIndex+'_'+trialtype+'_'+ str(trialIndex) +'_'+participant+'_'+word+'_'+modality+correction+'_video_raw'+'.avi'
        # codec for the video
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        # get the frame width
        frameWidth = capture.get(cv2.CAP_PROP_FRAME_WIDTH)
        # get the frame height
        frameHeight = capture.get(cv2.CAP_PROP_FRAME_HEIGHT)

        # start writing video
        print('Starting to write the video')
        #out = ffmpegcv.VideoWriter(vidloc, fourcc, fps = originalfps, frameSize = (int(frameWidth), int(frameHeight)))
        out = cv2.VideoWriter(vidloc, fourcc, fps = originalfps, frameSize = (int(frameWidth), int(frameHeight)))

        print('Looping over frames')

        # index the frames of the current trial
        for fra in frames:
            capture.set(cv2.CAP_PROP_POS_FRAMES, fra)
            ret, frame = capture.read()
            if ret:
                out.write(frame)
            if not ret:
                print('a frame was dropped: ' + str(fra))
            
        capture.release()
        out.release()
        
        print('Video is done')

print('All done!')


Now we also write the video of tpose, using the same code but looking only for csv files with tpose in the name


In [ ]:
errorlist = []

# this is a folder with timeseries
tsfolder = datafolder+'Data_processed\Data_trials\\'
# this is where the raw long video is
videofolder = datafolder+'raw_data\\'
#print(tsfolder)
#print(videofolder)

# loop through the csv's in tsfolder that has string 'MyWebcamStream' in name
for file in os.listdir(tsfolder):
    if 'MyWebcamFrameStream' in file and 'tpose' in file:
        print('Now processing file '+file)
        print(file)
        # get the filename 
        filename = file.split('.')[0]
        sessionIndex = filename.split('_')[0] + '_' + filename.split('_')[1]

        videolong = videofolder+sessionIndex+'\\'+sessionIndex+'-video.avi' # this is the long video

        begin_time = trialdata['0'].min() # begin time of the trial
        #print(begin_time)
        end_time = trialdata['0'].max() # end time of the trial
        #print(end_time)
        # get the begin and end frame
        begin_frame = trialdata['1'].min().astype(int)
        #print(begin_frame)
        end_frame = trialdata['1'].max().astype(int)
        #print(end_frame)
        totframes = end_frame-begin_frame # total number of frames in the trial
        #print(totframes)
        frames = range(begin_frame, end_frame) #get all the frames in trial
        #print(frames)
        
        #load in the long video
        print('Loading the original video')
        capture = cv2.VideoCapture(videolong) 

        originalfps = round((totframes/(end_time-begin_time)),3)
        print('original fps: '+str(originalfps))
        
        # this is the location where the video will be saved
        vidloc = trialfolder+filename+'_video_raw'+'.avi'
        # codec for the video
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        # get the frame width
        frameWidth = capture.get(cv2.CAP_PROP_FRAME_WIDTH)
        # get the frame height
        frameHeight = capture.get(cv2.CAP_PROP_FRAME_HEIGHT)
        # start writing video
        print('Starting to write the video')
        #out = ffmpegcv.VideoWriter(vidloc, fourcc, fps = originalfps, frameSize = (int(frameWidth), int(frameHeight)))
        out = cv2.VideoWriter(vidloc, fourcc, fps = originalfps, frameSize = (int(frameWidth), int(frameHeight)))

        print('Looping over frames')

        # index the frames of the current trial
        for fra in frames:
            capture.set(cv2.CAP_PROP_POS_FRAMES, fra)
            ret, frame = capture.read()
            if ret:
                out.write(frame)
                
            if not ret:
                print('a frame was dropped: ' + str(fra))
            
        capture.release()
        out.release()

print('All done!')


# Concatenating audio and video, using ffmpeg


In [ ]:
import ffmpeg

wavloc = trialfolder+'Audio\\'
audiovideo = datafolder+'Data_processed\\AudioVideo\\'

if not os.path.exists(wavloc):
    print(f"Directory not found: {wavloc}")

# loop over Audio files
for file in os.listdir(wavloc):
    print(file)
    if 'denoised' in file and '0_1' in file:
        print('Now processing file '+file)
        dyadIndex = file.split('_')[0]   # this is dyad number
        partIndex = file.split('_')[1]   # this is part of the session
        sessionIndex = dyadIndex + '_' + partIndex # this is the session index
        trialIndex = file.split('_')[3] # this is trial number

        participant = file.split('_')[7] # this is participant 0/1
        word = file.split('_')[8] # this is the word
        modality = file.split('_')[9] # this is the modality
        # put the .csv away from the modality
        modality = modality.split('.')[0]

        if 'c0' in file:
            correction = '_c0'
        elif 'c1' in file:
            correction = '_c1'
        elif 'c2' in file:
            correction = '_c2'
        else:
            correction = ''
        
        if 'pr' in file:
            trialtype = 'pr'
        else:
            trialtype = 'trial'
        #load in the audio
        print('Loading the audio')
        audio_path = os.path.join(wavloc, file)
        if not os.path.exists(audio_path):
            print(f"Audio file not found: {audio_path}")
        # input the video with ffmpg
        input_audio = ffmpeg.input(audio_path)
        print(input_audio)
        #load in the video with matchich trialIndex and SessionIndex
        print('Loading the video')
        video_path = os.path.join(trialfolder, f"{sessionIndex}_{trialtype}_{trialIndex}_{participant}_{word}_{modality}{correction}_video_raw.avi")

        if not os.path.exists(video_path):
            print(f"Video file not found: {video_path}")
        input_video = ffmpeg.input(video_path)
        print(input_video)
        # get information about the vid_frate
        #streamloc = trialfolder+sessionIndex+'_trial_'+ str(trialIndex) +'_'+'MyWebcamFrameStream_nominal_srate500'+'.csv'
        #print(streamloc)
        #streamdata = pd.read_csv(streamloc)
        # get the begin and end frame
        #begfr = streamdata['1'].min().astype(int)
        #print(begfr)
        #endfr = streamdata['1'].max().astype(int)
        #print(endfr)
        #totfr = endfr-begfr
        #print(totfr)
        #begin = streamdata['0'].min()
        #print(begin)
        #end = streamdata['0'].max()
        #print(end)
        # what is the original fps of the video
        #origfps = round((totfr/(end-begin)),3)
        # tranform it into real number
        #origfps = float(origfps)
        
        #combine the audio and video
        print('Combining audio and video')
        output_path = os.path.abspath(os.path.join(audiovideo, f"{sessionIndex}_{trialtype}_{trialIndex}_{participant}_{word}_{modality}{correction}_final.avi"))
        ffmpeg.concat(input_video, input_audio, v=1, a=1).output(output_path).run(overwrite_output=True)
        
        #save it
        print('Saving the video')
        #print(origfps)
        #print(type(origfps))
        #print(trialIndex)
        #print(sessionIndex)
        # save the final video with audio
        #final.write_videofile(trialfolder+sessionIndex+'_trial_'+ str(trialIndex) +'_'+'video_audio'+'.mp4', fps=origfps)

In [ ]:
# # open prompt and ask for id
# id = input("Enter ID: ")
# # create a folder with name of the id


# # select an output path that will have a folderts from 1 to 1000, and if th folder does not exist, crate it
# output_path = os.path.abspath(os.path.join(datafolder, f"Data_processed\\{id}"))